In [1]:
import semantic_kernel as sk

kernel = sk.Kernel()

from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

api_key, org_id = sk.openai_settings_from_dot_env()
service_id = "default"
kernel.add_service(
    OpenAIChatCompletion(service_id=service_id, ai_model_id="gpt-3.5-turbo-0125", api_key=api_key, org_id=org_id),
)

plugin = kernel.import_plugin_from_prompt_directory("./Plugins/", "ReportPlugin")
OutlineJSON_function = plugin["OutlineJSON"]
OutlineJSON = await kernel.invoke(OutlineJSON_function, sk.KernelArguments(input="Báo cáo thị trường Tiêu dùng năm 2023"))
print(OutlineJSON)



{'report': {'title': 'Báo cáo thị trường Tiêu dùng năm 2023','introduction': 'Báo cáo này sẽ đánh giá tình hình thị trường tiêu dùng trong năm 2023, bao gồm xu hướng tiêu dùng, sự thay đổi trong hành vi mua sắm của người tiêu dùng, và ảnh hưởng của các yếu tố kinh tế và xã hội.','body': {'headings': ['Xu hướng tiêu dùng','Hành vi mua sắm của người tiêu dùng','Yếu tố ảnh hưởng đến thị trường tiêu dùng']}, 'conclusion': 'Tóm lại, thị trường tiêu dùng năm 2023 đang trải qua những biến động đáng chú ý và cần sự chú ý đến các yếu tố ảnh hưởng.','recommendations': 'Dựa trên những phân tích trong báo cáo, chúng tôi đề xuất các chiến lược tiếp thị và kinh doanh phù hợp để thích nghi với thị trường tiêu dùng năm 2023.'}}


In [2]:
from Plugins.utils import json_split
outline_dict = json_split(str(OutlineJSON))
outline_dict

{'title': 'Báo cáo thị trường Tiêu dùng năm 2023',
 'introduction': 'Báo cáo này sẽ đánh giá tình hình thị trường tiêu dùng trong năm 2023, bao gồm xu hướng tiêu dùng, sự thay đổi trong hành vi mua sắm của người tiêu dùng, và ảnh hưởng của các yếu tố kinh tế và xã hội.',
 'body': {'headings': ['Xu hướng tiêu dùng',
   'Hành vi mua sắm của người tiêu dùng',
   'Yếu tố ảnh hưởng đến thị trường tiêu dùng']},
 'conclusion': 'Tóm lại, thị trường tiêu dùng năm 2023 đang trải qua những biến động đáng chú ý và cần sự chú ý đến các yếu tố ảnh hưởng.',
 'recommendations': 'Dựa trên những phân tích trong báo cáo, chúng tôi đề xuất các chiến lược tiếp thị và kinh doanh phù hợp để thích nghi với thị trường tiêu dùng năm 2023.'}

In [3]:
Introduction_function = plugin["Introduction"]
Introduction = await kernel.invoke(Introduction_function, sk.KernelArguments(introduction=outline_dict["introduction"]
                                                                            , reportLayout = str(OutlineJSON)))
print(Introduction)

Báo cáo thị trường Tiêu dùng năm 2023 sẽ cung cấp một cái nhìn tổng quan về tình hình thị trường tiêu dùng trong năm tới, tập trung vào việc đánh giá các xu hướng tiêu dùng đáng chú ý, sự thay đổi trong hành vi mua sắm của người tiêu dùng, cũng như ảnh hưởng của các yếu tố kinh tế và xã hội đối với thị trường. Bằng việc phân tích các thông tin này, chúng ta sẽ có cái nhìn sâu hơn về cách thị trường tiêu dùng đang phát triển và những thách thức mà các doanh nghiệp có thể phải đối mặt.


In [4]:
from Plugins.google_search import GoogleSearchPlugin
from Plugins.utils import token_count
import asyncio
TOKEN_LIMIT = 10000

google_plugin = kernel.import_plugin_from_object(GoogleSearchPlugin(), plugin_name = "GoogleSearchPlugin")
google_search_function = google_plugin["GoogleSearch"]
get_content_function = google_plugin["GetContentFromURL"]

column_function = plugin["Column"]
column_summ_function = plugin["ColumnSumm"]
headings = outline_dict["body"]["headings"]
query_function = plugin["Query"]

async def columnist(heading, title):
    search_result = []
    tokens = 0
    query = str(await kernel.invoke(query_function, sk.KernelArguments(input = heading, big_title = title)))
    search_result_urls = await kernel.invoke(google_search_function, sk.KernelArguments(keyword = query, num_results = 15))
    search_result_urls = search_result_urls.value
    print(search_result_urls)
    while True:
        if not search_result_urls:
            break
        url = search_result_urls.pop(0)
        content = await kernel.invoke(get_content_function, sk.KernelArguments(url = url))
        content = content.value
        new_tokens = token_count(content)
        if tokens + new_tokens > TOKEN_LIMIT:
            print(tokens)
            if tokens >= TOKEN_LIMIT//2:
                break
            else:
                continue
        tokens += new_tokens
        search_result.append(content)
    search_result = " ".join(search_result)
    column = await kernel.invoke(column_function, sk.KernelArguments(input = search_result, body_content = heading))
    column_summ = await kernel.invoke(column_summ_function, sk.KernelArguments(input = search_result, body_content = heading))
    return str(column), str(column_summ)

coros = []
for heading in headings:
    coros.append(columnist(heading, outline_dict["title"]))

values = await asyncio.gather(*coros)
print(values)




['https://pmax.com.vn/blog/xu-huong-thi-truong-va-nguoi-tieu-dung-nua-cuoi-2023-dau-2024/#Cuoi_nam_la_mua_mua_sam_cao_diem_nhat_trong_nam', 'https://www.cimigo.com/vi/trends/xu-huong-tieu-dung-viet-nam-2023/', 'https://innovativehub.com.vn/tong-quan-va-xu-huong-nganh-hang-tieu-dung-viet-nam-2023/', 'https://pmax.com.vn/blog/xu-huong-thi-truong-va-nguoi-tieu-dung-nua-cuoi-2023-dau-2024/', 'https://advertisingvietnam.com/10-xu-huong-hanh-vi-nguoi-tieu-dung-nam-2023-p22638', 'https://advertisingvietnam.com/xu-huong-thi-truong-nguoi-tieu-dung-nua-cuoi-2023-dau-2024-p22690', 'https://pmax.com.vn/blog/xu-huong-thi-truong-va-nguoi-tieu-dung-nua-cuoi-2023-dau-2024/#Tong_quan_tinh_hinh_kinh_te_6_thang_dau_nam_2023', 'https://pmax.com.vn/blog/xu-huong-thi-truong-va-nguoi-tieu-dung-nua-cuoi-2023-dau-2024/#Xu_huong_thi_truong', 'https://glints.com/vn/blog/xu-huong-hanh-vi-nguoi-tieu-dung/', 'https://aglobal.vn/blog/xu-huong-tieu-dung-quy-4-nam-2023-nhung-thay-doi-can-luu-y-1564161623', 'https://vn

In [5]:
for value in values:
    print(token_count(value[0]), "tokens")
    print(value[0])
    print(token_count(value[1]), "tokens")
    print(value[1])

1126 tokens
Xu hướng tiêu dùng là một phần quan trọng của nền kinh tế và thị trường tiêu dùng hiện nay. Trong bối cảnh nền kinh tế Việt Nam đang phải đối mặt với nhiều khó khăn và thách thức, việc hiểu rõ xu hướng tiêu dùng là điều cực kỳ quan trọng để các doanh nghiệp có thể thích nghi và phát triển trong môi trường cạnh tranh gay gắt.

Theo những thông tin được cung cấp, chúng ta thấy rằng tình hình kinh tế Việt Nam trong 6 tháng đầu năm 2023 đã có những dấu hiệu tích cực. Tăng trưởng GDP tăng lên 4,14% trong quý 2, và lạm phát được kiểm soát. Tuy nhiên, vẫn còn những biến động trong giá cả và các yếu tố khác ảnh hưởng đến hành vi tiêu dùng của người dân.

Dự báo cho nửa sau năm 2023 cho thấy rằng kinh tế Việt Nam có khả năng phục hồi mạnh mẽ, với dự đoán tăng trưởng GDP đạt 7%. Điều này sẽ tạo ra một môi trường kinh doanh tích cực và đầy triển vọng cho các doanh nghiệp.

Trên thị trường tiêu dùng, cuối năm là thời điểm mua sắm cao điểm nhất trong năm. Các sự kiện mua sắm lớn như Bla

In [6]:
conclusion_function = plugin["Conclusion"]
summContent = " ".join([value[1] for value in values])
conclusion = await kernel.invoke(conclusion_function, sk.KernelArguments(conclusion = outline_dict["conclusion"], summContent = summContent))
print(str(conclusion))


Tổng kết lại, thị trường tiêu dùng của Việt Nam trong năm 2023 đang chứng kiến những thay đổi đáng chú ý và đòi hỏi sự quan tâm đến các yếu tố ảnh hưởng. Với tình hình kinh tế đang phục hồi và triển vọng tăng trưởng GDP tích cực, người tiêu dùng đã bắt đầu có những điều chỉnh trong hành vi mua sắm và quyết định tiêu dùng. Xu hướng thị trường cho thấy sự tăng cường mua sắm trực tuyến, quan tâm đến sản phẩm bền vững và trải nghiệm mua sắm chất lượng.

Hành vi mua sắm của người tiêu dùng được thúc đẩy bởi sự lạc quan về tương lai kinh tế và nhu cầu cơ bản, đồng thời họ ngày càng thông minh trong việc so sánh và lựa chọn sản phẩm. Xu hướng mua sắm trong năm 2024 dự kiến sẽ chứng kiến sự phát triển của công nghệ mới và ảnh hưởng mạnh mẽ từ Influencer Marketing.

Để thành công trên thị trường tiêu dùng hiện nay, các doanh nghiệp cần nắm bắt kịp thời những xu hướng và insights mới, đồng thời áp dụng công nghệ và dữ liệu để hiểu rõ hơn về khách hàng. Việc đáp ứng nhanh chóng và hiệu quả vào nh

In [7]:
print(token_count(str(conclusion)))

607


In [8]:
recommendation_function = plugin["Recommendation"]
summContent = " ".join([value[1] for value in values])
recommendation = await kernel.invoke(recommendation_function, sk.KernelArguments(recommendation = outline_dict["recommendations"], summContent = summContent))
print(str(recommendation))
print(token_count(str(recommendation)))

Dựa trên những phân tích trong báo cáo, chúng tôi đề xuất các chiến lược tiếp thị và kinh doanh phù hợp để thích nghi với thị trường tiêu dùng năm 2023. Trước hết, chúng ta cần tập trung vào việc phát triển chiến lược tiếp thị trực tuyến hiệu quả để đáp ứng nhu cầu ngày càng tăng của người tiêu dùng trong việc mua sắm trực tuyến. Việc tối ưu hóa trải nghiệm mua sắm trực tuyến, cung cấp ưu đãi hấp dẫn và chăm sóc khách hàng chuyên nghiệp sẽ giúp tăng cường sự hài lòng và trung thành của khách hàng.

Thứ hai, chúng ta cần đầu tư vào việc phát triển sản phẩm bền vững và thân thiện với môi trường để đáp ứng xu hướng ngày càng tăng của người tiêu dùng trong việc chọn lựa sản phẩm. Việc tạo ra các sản phẩm có giá trị và chất lượng cao, đồng thời chú trọng đến việc bảo vệ môi trường sẽ giúp thương hiệu xây dựng uy tín và lòng tin từ phía khách hàng.

Thứ ba, việc tận dụng công nghệ mới như trải nghiệm mua sắm thực tế ảo và thực tế tăng cường, cũng như sử dụng trí tuệ nhân tạo để cải thiện trả

In [9]:
for value in values:
    print(token_count(value[0]), "tokens")
    print(value[0])

1126 tokens
Xu hướng tiêu dùng là một phần quan trọng của nền kinh tế và thị trường tiêu dùng hiện nay. Trong bối cảnh nền kinh tế Việt Nam đang phải đối mặt với nhiều khó khăn và thách thức, việc hiểu rõ xu hướng tiêu dùng là điều cực kỳ quan trọng để các doanh nghiệp có thể thích nghi và phát triển trong môi trường cạnh tranh gay gắt.

Theo những thông tin được cung cấp, chúng ta thấy rằng tình hình kinh tế Việt Nam trong 6 tháng đầu năm 2023 đã có những dấu hiệu tích cực. Tăng trưởng GDP tăng lên 4,14% trong quý 2, và lạm phát được kiểm soát. Tuy nhiên, vẫn còn những biến động trong giá cả và các yếu tố khác ảnh hưởng đến hành vi tiêu dùng của người dân.

Dự báo cho nửa sau năm 2023 cho thấy rằng kinh tế Việt Nam có khả năng phục hồi mạnh mẽ, với dự đoán tăng trưởng GDP đạt 7%. Điều này sẽ tạo ra một môi trường kinh doanh tích cực và đầy triển vọng cho các doanh nghiệp.

Trên thị trường tiêu dùng, cuối năm là thời điểm mua sắm cao điểm nhất trong năm. Các sự kiện mua sắm lớn như Bla

In [10]:
import semantic_kernel as sk

kernel = sk.Kernel()

from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

api_key, org_id = sk.openai_settings_from_dot_env()
service_id = "default"
kernel.add_service(
    OpenAIChatCompletion(service_id=service_id, ai_model_id="gpt-3.5-turbo-0125", api_key=api_key, org_id=org_id),
)

plugin = kernel.import_plugin_from_prompt_directory("./Plugins/", "ReportPlugin")
Query_function = plugin["Query"]
Query = await kernel.invoke(Query_function, sk.KernelArguments(input="Tình hình hiện tại", big_title = "Báo cáo thị trường Tiêu dùng năm 2023"))
print(Query)


"Tình hình hiện tại thị trường tiêu dùng Việt Nam năm 2023"
